In [1]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from itertools import combinations
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFE
from bayes_opt import BayesianOptimization

In [2]:
data=pd.read_excel(r"C:\Users\HP\Desktop\Data.xlsx",sheet_name='16+3',index_col=0,header=0)
data1=data.iloc[0:16, :]

minmax_scaler=preprocessing.MinMaxScaler()  
data2=minmax_scaler.fit_transform(data1)
data3=pd.DataFrame(data2,columns=['O3_input','lg(O3)','H2O2_input','lg(H2O2)','H2O2/O3','lg(H2O2/O3)','pH','volumn',
                                  'TOC','UV254','Φn1','Φn2','Φn3','Φn4','Φn5','FRI','FRI20','OU20','OU60','OU120',
                                 'O3(1)20','O3(1)60','O3(1)120','O3(g)20','O3(g)60','O3(g)120','H2O2_20','H2O2_60',
                                  'H2O2_120','FMax1','FMax2','FMax3','FMax4','cost'])
data4=data3[['lg(O3)','lg(H2O2)','pH','TOC','UV254','Φn1','Φn2','Φn3','Φn4','Φn5','FRI','OU20','OU60','OU120',
                                 'O3(1)20','O3(1)60','O3(1)120','O3(g)20','O3(g)60','O3(g)120','H2O2_20','H2O2_60',
                                  'H2O2_120','FMax1','FMax2','FMax3','FMax4']]

,O3_input,lg(O3),H2O2_input,lg(H2O2),H2O2/O3,lg(H2O2/O3),pH,volumn,TOC,UV254,...,O3(g)60,O3(g)120,H2O2_20,H2O2_60,H2O2_120,FMax1,FMax2,FMax3,FMax4,cost
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.174272,0.276007,...,0.003091,0.003882,0.022302,0.041805,0.000000,0.291770,0.338220,0.197112,0.346430,0.000000
1,0.000000,0.000000,0.019382,0.469188,1.000000,1.000000,0.333333,0.0,0.049316,0.130054,...,0.002041,0.001648,0.127310,1.000000,0.135055,0.261372,0.179775,0.334419,0.153176,0.050139
2,0.000000,0.000000,0.002931,0.234969,0.151232,0.500800,0.666667,0.0,0.085820,0.000000,...,0.003283,0.003787,0.026079,0.051821,0.000167,0.000000,0.000000,0.000000,0.000000,0.013243
3,0.000000,0.000000,0.019382,0.469188,1.000000,1.000000,1.000000,0.0,0.000000,0.495418,...,0.000000,0.000000,0.040433,0.207068,0.022114,0.570154,0.465007,0.624370,0.519317,0.124692
4,0.183673,0.588592,0.005673,0.312432,0.000000,0.000000,0.000000,0.0,0.094595,0.519975,...,0.017796,0.022048,0.016258,0.045143,0.006895,0.793536,0.648313,0.799742,0.747103,0.371966
5,0.183673,0.588592,0.005673,0.312432,0.000000,0.000000,0.333333,0.0,0.587224,0.564706,...,0.165586,0.181925,0.005870,0.518828,0.074685,0.996455,1.000000,1.000000,0.983669,0.102126
6,0.183673,0.588592,0.034985,0.547401,0.151232,0.500800,0.333333,0.0,0.696736,0.607049,...,0.146710,0.162042,0.007638,0.799496,0.118716,0.974190,0.950282,0.987169,0.969692,0.095574
7,0.183673,0.588592,0.116579,0.709029,0.572205,0.845285,0.333333,0.0,0.803263,1.000000,...,0.083098,0.085491,0.111194,0.150287,0.009459,0.994320,0.975092,0.980770,0.979919,0.109757
8,0.183673,0.588592,0.199496,0.781620,1.000000,1.000000,0.333333,0.0,0.548614,0.848700,...,0.049286,0.052538,0.542431,0.647771,0.031726,0.954510,0.925464,0.955066,0.900332,0.197980
9,0.183673,0.588592,0.199496,0.781620,1.000000,1.000000,0.666667,0.0,0.559319,0.835121,...,0.038794,0.041300,0.127864,0.477572,0.017636,0.981925,0.984824,0.981197,0.979841,0.192236


In [23]:
data7 = data4.values
column_names = data4.columns

X_original = data7[:, [i for i in range(data7.shape[1]) if i!= 3]]
y_original = data7[:, 3]

def MODEL(X, y):
    
    poly=PolynomialFeatures(degree=2,interaction_only=True)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    loo = LeaveOneOut()
    y_real = []
    y_predicted = []

    # Applying LOO-CV
    for train_index, test_index in loo.split(X_poly):
        X_train, X_test = X_poly[train_index], X_poly[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test)
        y_real.append(y_test[0])
        y_predicted.append(y_pred[0])
    res = r2_score(y_real, y_predicted)
    return res


initial_features = X_original[:, :3]


remaining_indices = range(3, X_original.shape[1])
all_combinations = list(combinations(remaining_indices, 2))


all_results = []


for comb in all_combinations:

    new_feature_subset = np.hstack((initial_features,
                                    X_original[:, comb[0]].reshape(-1, 1),
                                    X_original[:, comb[1]].reshape(-1, 1)))


    res = MODEL(new_feature_subset, y_original)


    all_results.append((res, new_feature_subset, [0, 1, 2] + list(comb)))


all_results.sort(key=lambda x: x[0], reverse=True)

# Outputs the top 5 highest maxx2 values, corresponding feature subsets and feature indexes.
for i in range(5):
    print(f"top {i + 1}maxx2 value:", all_results[i][0])

    data7_indexes = [index if index < 3 else index + 1 for index in all_results[i][2]]

    print(f"The corresponding feature subset:", all_results[i][1])
    print(f"The corresponding index in data7:", data7_indexes)
    print(f"The corresponding variable name:", [column_names[index] for index in data7_indexes])

第1高的maxx2值: 0.998159928575748
对应的特征子集: [[0.         0.         0.33333333 0.04180459 0.3464303 ]
 [0.         0.46918772 0.33333333 1.         0.15317608]
 [0.         0.23496924 0.66666667 0.05182057 0.        ]
 [0.         0.46918772 1.         0.20706826 0.51931682]
 [0.58859191 0.31243181 0.         0.04514325 0.74710311]
 [0.58859191 0.31243181 0.33333333 0.51882828 0.98366912]
 [0.58859191 0.54740106 0.33333333 0.79949562 0.96969237]
 [0.58859191 0.70902911 0.33333333 0.15028726 0.9799193 ]
 [0.58859191 0.78161954 0.33333333 0.64777137 0.9003323 ]
 [0.58859191 0.78161954 0.66666667 0.47757205 0.97984117]
 [0.58859191 0.68790635 0.66666667 0.31129697 0.98361018]
 [0.58859191 0.31243181 1.         0.         0.95808395]
 [0.58859191 0.78161954 1.         0.01509531 0.96131148]
 [1.         1.         0.33333333 0.70010563 0.99993918]
 [1.         0.53081228 0.66666667 0.         0.97692839]
 [1.         0.76578152 1.         0.         1.        ]]
对应的data7中的索引: [0, 1, 2, 13, 18]
